## Imports

In [12]:
import numpy as np

## Load data

In [13]:
#rawdata = np.loadtxt("data/data_OHLC.txt", delimiter=",")
data_raw = np.genfromtxt("data/data_OHLC.txt", delimiter=",")
print(data_raw[0])

In [15]:
# drop date first column
data_noT = data_raw[:,1:]
print(data_noT[0])

[4196.75 4200.25 4196.25 4197.5 ]
